# Padaria Portugesa Geolocation

In [3]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon, Point
import geopandas as gp
import geoplot as gplt

In [4]:
freguesias = gp.read_file('data/freguesias.shp')

DriverError: data/freguesias.shp: No such file or directory